In [4]:
import pickle
from tkinter import *
import numpy as np
import pandas as pd
import csv
from fuzzywuzzy import fuzz
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

tr_df=pd.read_csv("diab.csv")

tr_df["output"] = tr_df["output"].replace(['No_Diabetes','Prediabetis','Type1','Type2'],[0,1,2,3])

tr_X=tr_df.iloc[:,:-1]
tr_y=tr_df.iloc[:,-1:]

tr_Xtrain,tr_Xtest,tr_ytrain,tr_ytest = train_test_split(tr_X,tr_y,test_size = 0.2,random_state = 0)

d_train = lgb.Dataset(tr_Xtrain, label=tr_ytrain)

params = {}
params['learning_rate'] = 0.05
params['boosting_type'] = 'gbdt'
params['objective'] = 'multiclass'
params['metric'] = 'multi_logloss'
params['num_class'] = 4
params['sub_feature'] = 0.5
params['num_leaves'] = 200
params['min_data'] = 10
params['max_depth'] = 20
    
clf = lgb.train(params, d_train, 500)

pickle.dump(clf, open('ml_model.sav','wb'))



In [5]:
def prediction_model():
    tr_pred = clf.predict(tr_Xtest)
    accuracy = accuracy_score(tr_ytest, np.argmax(tr_pred, axis = 1))*100
    print("LightGBM Model accuracy=",accuracy)

    X=pd.read_csv("diabres.csv")
    with open('diabres.csv',newline='') as f:
        reader = csv.reader(f)
        data = list(reader)
    
    X_new= data
    #X_new = [[6.4,120,6.2]]
    #X_new = [[29.5,300,28.75]]
    Y_new = clf.predict(X_new)

    y_new = np.argmax(Y_new, axis = 1)
    print(y_new)

    if(y_new==0):
        print('No_Diabetes')
    elif(y_new==1):
        print('Prediabetes')
    elif(y_new==2):
        print('Type1')
    else:
        print('Type2')
    return y_new

In [6]:
prediction_model()

LightGBM Model accuracy= 99.92301770592763
[1]
Prediabetes


array([1], dtype=int64)